In [63]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain_core.prompts import load_prompt
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

from datetime import datetime
from urllib.parse import urlencode
from typing_extensions import TypedDict
from typing import List, Annotated, AsyncGenerator
from langgraph.graph.message import add_messages



In [57]:
model_name = "gemma3:12b"
ollama_url = "http://192.168.50.70:11434"
prompt_path = "/Users/netager/Docker_Data/openwebui-dify/rag_data/prompts/law.yaml"
model_path = "/Users/netager/Docker_Data/openwebui-dify/rag_data/HUGGING_FACE_MODEL/BAAI_bge-m3"
chroma_path = "/Users/netager/Docker_Data/openwebui-dify/rag_data/Chroma_DB/chroma_bank_law_db"
k = 3

In [64]:
class State(TypedDict):
    question: Annotated[str, "Question"]  # 질문(누적되는 list)
    context: Annotated[str, "Context"]  # 문서의 검색 결과
    answer: Annotated[str, "Answer"]  # 답변
    messages: Annotated[list, add_messages]  # 메시지(누적되는 list)
    relevance: Annotated[str, "Relevance"]  # 관련성

In [ ]:
def create_embedding():
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
        encode_kwargs={"normalize_embeddings": True},
    )

def create_vectorstore():
    return Chroma(
            persist_directory=chroma_path,
            embedding_function=create_embedding(),
            collection_name="bank_law_case",
        )

def create_retriever(vectorstore):
    return vectorstore.as_retriever(
        search_type="similarity", 
        search_kwargs={"k": k},

        # search_type="similarity_score_threshold", 
        # search_kwargs={"k": k, "score_threshold": 0.2},
        # Add mmr search for diversity
        # search_type="mmr",
        # search_kwargs={"k": 1, "fetch_k": 3, "lambda_mult": 0.5}
    )

def create_model():
    # return ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
    return ChatOllama(
            # base_url='http://host.docker.internal:11434',
            base_url=ollama_url,
            model=model_name,
            temperature=0,
            context_window=8192,
        )

def create_prompt():
    # return hub.pull("teddynote/rag-prompt-chat-history")
    return load_prompt(prompt_path)

def create_chain():
    vectorstore = create_vectorstore()
    retriever = create_retriever(vectorstore)
    model = create_model()
    prompt = create_prompt()
    chain = (
        {
            "question": itemgetter("question"),
            "context": itemgetter("context"),
            "chat_history": itemgetter("chat_history"),
        }
        | prompt
        | model
        | StrOutputParser()
    )
    return chain

In [69]:
vectorstore = create_vectorstore()
print(f"Create Vectore Store: {datetime.now()}")

retriever = create_retriever(vectorstore)
print(f"Create Retriever: {datetime.now()}")

user_input = "전산운영위원회 위원에 대해 알려줘."
# search_results = vectorstore.similarity_search_with_score(
#     query=user_input, 
#     k=5,
# )

context = retriever.invoke(user_input)  # Return Annotated[list[Document]
print(f"Retriever Invoke: {datetime.now()}")


# 체인 생성
chain = create_chain()
print(f"Create Chain: {datetime.now()}")

response = chain.invoke(
    {
        "question": user_input,
        "context": context,
        # "chat_history": messages_to_history(state["messages"]),
        "chat_history": []
    }
)
print(f"Chain Invoke: {datetime.now()}")


# print(f"[langgraph_agent_jb][llm_answer()] context: {context}")
search_results = context
if not search_results:
    logger.warning("No relevant documents found in search results.")
        
linked_docs = []
base_url = "https://jabis.jbbank.co.kr/jabis_pdf_view"

for search_result in search_results:
    # if search_result[1] < 0.8:  # relevance threshold
    params = {
        "source": search_result.metadata["source"],
        "title": search_result.metadata["title"],
        "page": search_result.metadata["page"] + 1,
    }
    url_with_params = base_url + "?" + urlencode(params)
    
    linked_docs.append(
        f"👉 [{params['title']}]({url_with_params}) [pages]: {params['page']}"
    )
    
response = response + "\n\n 📖 관련 문서 보기\n\n" + "\n\n".join(linked_docs)

print(response)

print(f"End Time: {datetime.now()}")


Create Vectore Store: 2025-06-23 17:50:22.736798
Create Retriever: 2025-06-23 17:50:22.737037
Retriever Invoke: 2025-06-23 17:50:22.951947
Create Chain: 2025-06-23 17:50:29.189316
Chain Invoke: 2025-06-23 17:50:51.053408
전북은행의 전산운영위원회 위원 구성에 대해 안내해 드리겠습니다. 관련 규정은 (D2009) 전산운영위원회 지침에 명시되어 있습니다.

**1. 위원 구성 (제2조)**

*   **상임위원:**
    *   전산담당임원
    *   종합기획부장
    *   인사지원부장
    *   마케팅기획부장
    *   준법감시부장
    *   IT기획부장
    *   IT개발부장
    *   정보보호부장
    *   디지털플랫폼부장
    *   *참고: 2022년 8월 8일 개정*
*   **위촉위원:** 심의사항 관련 부서 담당 본부장 및 부/실장 중에서 위원장이 선임

**2. 위원장 및 대행**

*   **위원장:** 전산담당본부장이 위원장이 되며, 위원장이 유고시 IT기획부장이 대행합니다. *참고: 2018년 7월 11일 개정*

**3. 실무협의회 구성**

*   실무협의회는 IT기획부장, IT기획부 기획팀장, 종합기획부, 인사지원부, 마케팅기획부, 준법감시부, IT개발부, 정보보호부, 디지털플랫폼부의 담당 책임자로 구성됩니다. *참고: 2022년 8월 8일 개정*
*   실무협의회의 의장은 IT기획부장, 부의장은 IT기획부 기획담당 부부장입니다. *참고: 2018년 7월 11일 개정*

**요약:** 전산운영위원회는 다양한 부서의 임원 및 담당자들이 상임위원으로 구성되며, 필요에 따라 외부 위촉위원도 참여합니다. 위원장은 전산담당본부장이 맡으며, IT기획부장이 대행 역할을 수행합니다. 실무협의회는 각 부서의 책임자들이 참여하여 전산 시스템 도입 사업 